In [5]:
import csv
import pandas as pd
import numpy as np
import cv2 
import glob, os, time, datetime
import timeit
from own_Functions import *
import matplotlib as mpl
import matplotlib.pyplot as plt


#This file creat an overlayer on top of the video showing when the types of turn, the detected position of the mouse, the number of reward remaining/the probability of reward/the number of reward obtained without leaving the object
#depending on the reward mode


#indicate the folder where your data are, the name of the animal and the session you which to obtained the video for
video_folder = '/localdata/alicedata/'
animalname='MOU3975'
session_name = 'MOU3975_20230318-1737'

#indicate which type of display correspond to the task of the mouse
reward_mode = "remaining" #the number on the object is how many reward remain in the oject
#reward_mode = "fixed" #the number on the object is the probability to get rewarded
#reward_mode = "linear_increase" #indicate the number of reward obtained without leaving this object

expert_mode = True #set to True if you want additional informations
ShowVideoWhileProcessing=False
ShowSubstractedVideo=False

#information on how to read the video. Take care that this two parameters are the same as they are for the recording
delaiBetweenVideoExperiment = 0 #number of frame before the start of the experiment #only for habitat/depleting protocols 

TRAPEZE_SIZE = 60 # we determine the width of the trapezes with TRAPEZE_SIZE


timeIntimout = 0 #time (in s) after the start of the experiment where the tracking should really commence

#this parameters help to see the true duration of exploration. However it only works with versions of exploration where the animal had to change object to stop it, and only if all object were rewarded.
exploration_time = 0 #used to show how long the mouse must explore after leaving an object 
###########################################################################################################################
video_name = session_name + ".avi"

def trapezes_from_patch(patch, width):
    """
    generate the trapezes coordinates surrounding a patch
    inputs:
    patch - coordinates of a patch [[Xa, Ya], [Xb, Yb], [Xc, Yc], [Xd, Yd]]
    width - width of the trapeze in pixels
    outputs:
    coordinates [[Xa, Ya], [Xb, Yb], [Xc, Yc], [Xd, Yd]] for the 4 trapezes.
    
    trapezes_from_patch(SWpatch_coords, 200)
    """
    # N = np.array([patch[0], patch[1], [patch[1][0]+width, patch[1][1]-width], [patch[0][0]-width, patch[0][1]-width]], np.int32).reshape((-1,1,2))
    # E = np.array([patch[1], patch[2], [patch[2][0]+width, patch[2][1]+width], [patch[1][0]+width, patch[1][1]-width]], np.int32).reshape((-1,1,2))
    # S = np.array([patch[2], patch[3], [patch[3][0]-width, patch[3][1]+width], [patch[2][0]+width, patch[2][1]+width]], np.int32).reshape((-1,1,2))
    # W = np.array([patch[3], patch[0], [patch[0][0]-width, patch[0][1]-width], [patch[3][0]-width, patch[3][1]+width]], np.int32).reshape((-1,1,2))
    N = [patch[0], patch[1], [patch[1][0]+width, patch[1][1]-width], [patch[0][0]-width, patch[0][1]-width]]
    E = [patch[1], patch[2], [patch[2][0]+width, patch[2][1]+width], [patch[1][0]+width, patch[1][1]-width]]
    S = [patch[2], patch[3], [patch[3][0]-width, patch[3][1]+width], [patch[2][0]+width, patch[2][1]+width]]
    W = [patch[3], patch[0], [patch[0][0]-width, patch[0][1]-width], [patch[3][0]-width, patch[3][1]+width]]

    return N, E, S, W

def reward_trapeze_change(current_line_turn_info, df, do_I_get_water, reward_mode, remaining_rewards, type_of_turn) : 
    if reward_mode == "remaining" :
        remaining_rewards = str(df.loc[df.index[current_line_turn_info], "maxNberOfConsecRewards"] - df.loc[df.index[current_line_turn_info], "nberOfConsecRewards"])
        if df.loc[df.index[current_line_turn_info], "typeOfTurn"] == "gogdet" or df.loc[df.index[current_line_turn_info], "typeOfTurn"] == "gogtet":
            type_of_turn = "gogtet"
            do_I_get_water = False
            remaining_rewards = "0"
        elif df.loc[df.index[current_line_turn_info], "typeOfTurn"] == "gobdet" or df.loc[df.index[current_line_turn_info], "typeOfTurn"] == "gobtet":
            type_of_turn = "gobtet"
            do_I_get_water = False
            remaining_rewards = "0"

    if df.loc[df.index[current_line_turn_info], "typeOfTurn"] == "expl" :
        type_of_turn = "expl"
        do_I_get_water = False
        remaining_rewards = "0"
    
    if (df.loc[df.index[current_line_turn_info], "typeOfTurn"] == "gogd" or df.loc[df.index[current_line_turn_info], "typeOfTurn"] == "gogt") and not df.loc[df.index[current_line_turn_info], "Rewarded"]:
        type_of_turn = "gogdnr"
        do_I_get_water = False
        remaining_rewards = "0"
    

    elif reward_mode == "linear_increase": #get the new number of consecutiv rewards around that object
        remaining_rewards = df.loc[df.index[current_line_turn_info], "nberOfConsecRewards"]
    
    elif reward_mode == "fixed" : #nothing to change when the reward mode is fixed
        pass

    return do_I_get_water, remaining_rewards, type_of_turn

##create in the MOU folder a new file with base name of the video to process
print('1')
print(session_name)
processed_video_folderPath=video_folder + animalname + '/' + session_name
print('2')
print(processed_video_folderPath)

# if glob.glob(processed_video_folderPath):
if os.path.isdir(processed_video_folderPath):
    print('processing folder already exist')
    input_video_path=processed_video_folderPath + '/' + video_name
    print('3')
    print(input_video_path)
    final_Video_path=input_video_path
else:
    input_video_path=video_folder + animalname + '/' + video_name
    final_Video_path=processed_video_folderPath + '/' + video_name
    #print(input_video_path)
    os.mkdir(processed_video_folderPath)

## the code will generate a processed video in which we can see the video tracking, region of interested visited, and some event (reward delivered etc ..)
## this video is saved as "sessionname_tracked.avi"

output_trackedVideo_name=video_name[:video_name.find('.')] + '_tracked' + video_name[video_name.find('.'):]
#print(output_trackedVideo_name)
output_trackedVideo_path=processed_video_folderPath + '/' + output_trackedVideo_name
#print(output_trackedVideo_path)


dfxyt = pd.read_csv(processed_video_folderPath + '/' + session_name + '_centroidTXY.csv')
#dfxyt = dfxyt.loc[dfxyt['time'] >= timeIntimout]
#get the number of the first frame past the time in Timout in the time that was recorded (which is different that the one obtained by the framerate)
index = dfxyt.loc[dfxyt['time'] >= timeIntimout].index[0]

#open turnsinfo to get the correct object and direction
df = pd.read_csv(processed_video_folderPath + '/' + session_name + '_turnsinfo.csv')
df=df.loc[df['time'] >= timeIntimout] # il y a des artefacts sur les premieres secondes de videos, donc il faut les supprimer
#df = df.append(df.iloc[-1])

current_line_turn_info = -1
indexDF = df.index.values[0] - 1
last_line = len(df.index) - 2

for i in range(indexDF + 1, df.index.values[-1]):#if there is a missing value for ongoingRewardedObject, replace it with either SW or SE, as long as it's not the one where the mouse is
    if (type(df['ongoingRewardedObject'][i]) == float) :
        df['ongoingRewardedObject'][i] = str([df['currentPatch'][i]]) 



ongoingRewardedObject = [a.strip("]['").split(', ') for a in df['ongoingRewardedObject']] #change the format of theses variables to be readable
ongoingRewardedObject = ongoingRewardedObject + 2 * [ongoingRewardedObject[-1]]
ongoingRewardedDirection = [a.strip('][').split(', ') for a in df['ongoingRewardedDirection']]
ongoingRewardedDirection = [['COUNTERCLOCKWISE',90] if len(i) == 1 and i[0] == '90' else ['CLOCKWISE', 270] if len(i) == 1 and i[0] == '270' else ['BOTH', 90, 270] for i in ongoingRewardedDirection ]
ongoingRewardedDirection.append(ongoingRewardedDirection[-1])



start_time = timeit.timeit()

# #Add input video directory here (ctrl+c on video and ctrl+v here to copy full directory)
# input_video='/home/david/Documents/Python/CorticoStriatalProject/NewCodesVideo/session_20221010-1243__SE.avi'

# output_video = input_video[:input_video.find('.')]+'_tracked'+input_video[input_video.find('.'):]


print('4')
print(input_video_path)

if glob.glob(input_video_path):
    print('Video found')
else :
    print('No video found, check input_video_path name')

good_object = ongoingRewardedObject[0][0] # with NE = object 3, NW = 4, SE = 1 and SW= 2
current_object = ongoingRewardedObject[0][0] #used to know where to plot
if reward_mode == "remaining" : 
    #remaining_rewards = str(df.iat[0, 12] - df.iat[0, 11])
    remaining_rewards = str(df.loc[df.index[0], "maxNberOfConsecRewards"] - df.loc[df.index[0], "nberOfConsecRewards"])
elif reward_mode == "fixed" :
    df_session = pd.read_csv(processed_video_folderPath + '/' + session_name + '_sessionparam.csv')
    remaining_rewards = df_session.loc[df_session.index[0], "proba" + ongoingRewardedObject[0][0]]
elif reward_mode == "linear_increase" : 
    remaining_rewards = df.loc[df.index[0], "nberOfConsecRewards"]
start = timeit.default_timer() #DAVQ: Take the time from computer clock. Will be used to substrsact time (t)



# find_go=input_video[input_video.find('__'):-3] # Finds automatically the correct object from the input video name (if follows the format from acquisition code)
# good_object = find_go[2:find_go.find('.')]

    #DAVQ: Are the zones equal to patches? 
    # Apparatus divided in 4 Zones: NW, NE, SW, SE as seen from above
    #
    #  ---------
    # ¦ NW ¦ NE ¦
    # ¦---------¦
    # ¦ SW ¦ SE ¦
    #  ---------

    # These are the [X, Y] coordinates of the 4 corners of the 4 patches. 
    # Each zone contains a patch. 
    #DAVQ: is the patch an object (4 walls)
    # A-------B
    # ¦       ¦
    # ¦       ¦
    # D-------C
    # Patch_coords = [[Xa, Ya], [Xb, Yb], [Xc, Yc], [Xd, Yd]]
    # From these coords, we can use the trapezes_from_patch() function to get the coordinates of the 4 zones surrounding each patch.
    # Width of trapezes is TRAPEZE_SIZE
    #       ---------------
    #      ¦ \           / ¦
    #      ¦   A-------B   ¦
    #      ¦   ¦       ¦   ¦
    #      ¦   ¦       ¦   ¦
    #      ¦   D-------C   ¦
    #      ¦ /           \ ¦
    #       ---------------
    # 



#Assign a number to each trapeze. We will use the difference between the value of the current trapeze and the preceding one to check in which direction
#the mouse is turning and if it follows the rule a reward will be given

current_direction = ongoingRewardedDirection[0][1:]   
direction = ongoingRewardedDirection[0][1:]  # direction to get rewards. Based  on function angleBetween. this vector -previsous vector
#direction = ['CLOCKWISE',270]

cardinalvectors= {'N': (0,1),'E': (1,0),'S': (0,-1),'W': (-1,0)} # those vectors point in the direction of the 4 reward ports from the center of the object
   
# open video
cap = cv2.VideoCapture(input_video_path) #Path define in the first lines
cap.set(cv2.CAP_PROP_POS_FRAMES, 0) # set the index of the next frame to be read to the one with index 0
# out of the box function to compute background
fgbg = cv2.createBackgroundSubtractorMOG2(varThreshold=16, detectShadows=True) # background elimination


# init some variables
# we're going to compare the positions of the mice at given frame versus the previous one. The previous one is noted with  "_" 
# and we need to initialise the previous one 
_x = _y = 0
_currentPatch = '',''
_currentTrapeze = currentTrapeze = 'none'
_rewarded = rewarded = 0.5  #this is the value we set when the mice is outstide the trapeze. (No reward will be delivered)
changedTrapeze = False
#change = datetime.datetime.now()    # change of patch. #Dav: looks like it is not used
change2 = datetime.datetime.now() # change of trapeze
#text = ""
time_save ,x_pos, y_pos = np.array([]) ,np.array([]) ,np.array([])  # Time and position x, y #DAVQ: MAybe consider change the name of the variable time to avoid confusion with python built in functions
speed , dist  , conseq , rew = np.array([]) ,np.array([]) ,np.array([]) ,np.array([]) # Used to compute distance at each delta(t), speed and the rewards
zone, trapeze = np.array([]) ,np.array([]) 

t , count_tot, count_conseq = 0, 0 ,0 #time will increment as 1/framerate,  count will +=1 for each reward
gtgo_c,gtgo_t, gtbo_c , gtbo_t , btgo_c,  btgo_t, btbo_c , btbo_t= np.array((0,0)) ,np.array((0,0)) ,np.array((0,0)) ,np.array((0,0)) ,np.array((0,0)) ,np.array((0,0)) ,np.array((0,0)) ,np.array((0,0)) 
btgo_count,gtgo_count,gtbo_count,btbo_count = 0, 0 ,0 ,0
exploring = True ; exploration_timer = 0; duration_last_exploration = 0
type_of_turn = "niao" ; couleur = (0, 255, 0) #initialized as " not in an object "

# Change to the framerate used for acquisition (i.e the input and output video should have the same duration)
resolution = 512,512   # number of pixel on the x and y axes        # Cannot get a higher resolution than the one we saved the video in (i.e can only degrade)
### changing the resolution will require to readjust all the dimensions of zones, objects, water port, trapezes. It is recommended to keep 512

framerate = int(cap.get(cv2.CAP_PROP_FPS))    # Get video framerate
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))-framerate #-framerate is used to avoid error when we arrive at the end of the video
#we should be able to do better. 

# save video
out = cv2.VideoWriter(output_trackedVideo_path, cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), framerate , (resolution))

#--------------------------------------------------
# position patches & trapezes & water ports
#--------------------------------------------------

#The patch coodinates in 512*512 pixel resolution. Real values 2048*2048 pixels images value after 4x bigger
#DavQ: not clear I though patches were the entire space divided in 4. Where is 0,0 .....
# NWpatch_coords = [[104, 125], [173, 125], [173, 201], [104, 201]]#value initialy in place in the code
# NEpatch_coords = [[330, 120], [400, 120], [400, 200], [330, 200]]
# SWpatch_coords = [[109, 351], [181, 351], [181, 418], [109, 418]]
# SEpatch_coords = [[330, 350], [400, 350], [400, 410], [330, 410]]

NWpatch_coords = [[104, 125], [173, 125], [173, 201], [104, 201]]#value is for Maud's acquisition code
NEpatch_coords = [[330, 120], [400, 120], [400, 200], [330, 200]]
SWpatch_coords = [[109, 351], [181, 351], [181, 410], [109, 410]]
SEpatch_coords = [[330, 350], [400, 350], [400, 410], [330, 410]]
# Resolution = 512,512
# NWpatch_coords = [[104, Resolution[1] -  125], [173, Resolution[1] -  125], [173, Resolution[1] -  201], [104, Resolution[1] -  201]]
# NEpatch_coords = [[330, Resolution[1] -  120], [400, Resolution[1] -  120], [400, Resolution[1] -  200], [330, Resolution[1] -  200]]
# SWpatch_coords = [[109, Resolution[1] -  351], [181, Resolution[1] -  351], [181, Resolution[1] -  410], [109, Resolution[1] -  410]]
# SEpatch_coords = [[330, Resolution[1] -  350], [400, Resolution[1] -  350], [400, Resolution[1] -  410], [330, Resolution[1] -  410]]



    
# Convert for use in openCV
patchNW = np.array(NWpatch_coords, np.int32).reshape((-1,1,2))
patchNE = np.array(NEpatch_coords, np.int32).reshape((-1,1,2))
patchSW = np.array(SWpatch_coords, np.int32).reshape((-1,1,2))
patchSE = np.array(SEpatch_coords, np.int32).reshape((-1,1,2))






# Positions of waterports (at the correct pixel number when resolution is 512*512, multiply by 4 each value to get in full 2048*2048 resolution)
waterports = {0:(138,121), 1:(182, 163), 2:(138, 205), 3:(96, 163),
                    4:(366, 121), 5:(408, 163), 6:(367, 206), 7:(324, 163),
                    8:(143, 347), 9:(185, 382), 10:(146, 424), 11:(102, 382),
                    12:(364, 348), 13:(404, 385), 14:(363, 426), 15:(322, 385)}
# display in red OFF ports
poly =  np.array([[20, 15], [20, 24], [29, 31], [44, 31], [51, 24], [51, 9], [44, 0], [29, 0]], np.int32)
inverserd_poly = np.array([[51, 15], [51, 24], [42, 31], [29, 31], [20, 24], [20, 9], [29, 0], [42, 0]], np.int32)#the origin is the top left, so use negative values in order to push where the top left of the figure will be
#points of the arrow drawing depending on the object
arrows_body = {"NW" : [(poly + (NWpatch_coords[0][0], int((NWpatch_coords[0][1] * 3 + NWpatch_coords[3][1])/4))).reshape((-1, 1, 2)), (inverserd_poly + (NWpatch_coords[0][0], int((NWpatch_coords[0][1] * 3 + NWpatch_coords[3][1])/4))).reshape((-1, 1, 2))],
            "NE" : [(poly + (NEpatch_coords[0][0], int((NEpatch_coords[0][1] * 3 + NEpatch_coords[3][1])/4))).reshape((-1, 1, 2)), (inverserd_poly + (NEpatch_coords[0][0], int((NEpatch_coords[0][1] * 3 + NEpatch_coords[3][1])/4))).reshape((-1, 1, 2))],
            "SW" : [(poly + (SWpatch_coords[0][0], int((SWpatch_coords[0][1] * 3 + SWpatch_coords[3][1])/4))).reshape((-1, 1, 2)), (inverserd_poly + (SWpatch_coords[0][0], int((SWpatch_coords[0][1] * 3 + SWpatch_coords[3][1])/4))).reshape((-1, 1, 2))],
            "SE" : [(poly + (SEpatch_coords[0][0], int((SEpatch_coords[0][1] * 3 + SEpatch_coords[3][1])/4))).reshape((-1, 1, 2)), (inverserd_poly + (SEpatch_coords[0][0], int((SEpatch_coords[0][1] * 3 + SEpatch_coords[3][1])/4))).reshape((-1, 1, 2))]}

reward_position = { "NW" : [133, 165],#coordinate on where to indicate a reward
                   "NE" : [360, 162],
                   "SW" : [140, 382],
                   "SE" : [360, 382]}


couleur_turn = {"gogt" : (101, 208, 134), "gogd" : (101, 208, 134), "gobt" : (50, 194, 241), "gobd" : (50, 194, 241), "bogt" : (56, 145, 230), "bogd" : (56, 145, 230), #indicate the color of the trapeze depending of the type of turn
                "bobd" : (0, 0, 204), "gogdet" : (224, 84, 119), "gobdet" : (136, 35, 239), "gogdnr" : (176, 39, 156),
                "bobt" : (0, 0, 204), "gogtet" : (224, 84, 119), "gobtet" : (136, 35, 239), "niao" : (241, 161, 73), "expl" : (241, 161, 73)}


# Processing loop, open video file, find animal position and see which path it took
# Saves all the parameters of interest (position, time, reward, speed ...) and plots a couple of graphs :
#2D and 3D path, reward frequency and types of interactions with the apparatus



# for i in range(50) :  #try to apply a higher level of background substraction in the first frames to get a quicker detection
#     ret, frm = cap.read() 

#     # resize the frame and the output 
#     frm = cv2.resize(frm, (resolution), interpolation = cv2.INTER_AREA) 
#     imgTrack = np.zeros_like(frm)

#     # convert to grayscale and add gaussian blur
#     frameGray = cv2.cvtColor(frm, cv2.COLOR_BGR2GRAY)  #can be commented if acquisition done in greyscale (it's the case when doing it with python script)
#     kernelSize = (25,25)
#     frameBlur = cv2.GaussianBlur(frameGray, kernelSize, 0) ## blurring removes details/noise

#     # apply the background substraction
#     thresh = fgbg.apply(frameBlur,learningRate=0.02) #learning rate == value between 0 and 1 that indicates how fast the background model is learnt. 0 means that the background model is not updated at all, 1 means that the background model is completely reinitialized from the last frame.
#     fgmask = cv2.resize(thresh, (resolution)) ## this is only used when display thresh

#     # compute center of mass of what's left and get x, y of the centroid
#     M = cv2.moments(thresh)
#     if M['m00'] == 0: continue
#     x = int(M['m10'] / M['m00'])
#     y = int(M['m01'] / M['m00'])
#     if _x == 0 and _y == 0:  #for the first frame now the previous frame take the current valu
#         _x = x
#         _y = y
        
#     #Saves timestamp and position in arrays for later use / save to .csv
#     t+=1/framerate
#     time=np.append(time,t)
#     x_pos=np.append(x_pos,x)
#     y_pos=np.append(y_pos,y)

#     if ShowVideoWhileProcessing:
#         cv2.imshow('frame', frm)  # final frame
#     # cv2.imshow('mask', fgmask)  # shape of the mouse as detected after processing

#     if ShowSubstractedVideo:
#         cv2.imshow("__", thresh)
    
#     out.write(frm) #saves frame to output video

#
for i in range(0, delaiBetweenVideoExperiment) :
    # get the current frame  
    ret, frm = cap.read()

    # resize the frame and the output 
    frm = cv2.resize(frm, (resolution), interpolation = cv2.INTER_AREA) 
    imgTrack = np.zeros_like(frm)

    # convert to grayscale and add gaussian blur
    frameGray = cv2.cvtColor(frm, cv2.COLOR_BGR2GRAY)  #can be commented if acquisition done in greyscale (it's the case when doing it with python script)
    kernelSize = (25,25)
    frameBlur = cv2.GaussianBlur(frameGray, kernelSize, 0) ## blurring removes details/noise

    # apply the background substraction
    thresh = fgbg.apply(frameBlur,learningRate=0.004) #learning rate == value between 0 and 1 that indicates how fast the background model is learnt. 0 means that the background model is not updated at all, 1 means that the background model is completely reinitialized from the last frame.
    fgmask = cv2.resize(thresh, (resolution)) ## this is only used when display thresh

    # compute center of mass of what's left and get x, y of the centroid
    M = cv2.moments(thresh)
    if M['m00'] == 0: continue
    x = int(M['m10'] / M['m00'])
    y = int(M['m01'] / M['m00'])
    if _x == 0 and _y == 0:  #for the first frame now the previous frame take the current valu
        _x = x
        _y = y
        
    #Saves timestamp and position in arrays for later use / save to .csv
    t+=1/framerate
    time_save=np.append(time_save,t)
    x_pos=np.append(x_pos,x)
    y_pos=np.append(y_pos,y)
    
    
    # plot a red dot at x, y 
    frm = cv2.circle(frm, (x,y), 7, (0,   0,   255), -1)
    thresh = cv2.circle(thresh, (x,y), 7, (255,   255,   255), -1)
    # plot the line between 2 successive frames and add it to frame
    imgTrack = cv2.addWeighted(np.zeros_like(imgTrack), 0.85, cv2.line(imgTrack, (x,y), (_x,_y), (255, 127, int(cap.get(cv2.CAP_PROP_POS_AVI_RATIO) * 255)), 2, cv2.LINE_AA), 0.98, 0.)
    frm = cv2.addWeighted(frm, 0.4, imgTrack, 1.0, 0.)

    out.write(frm) #saves frame to output video
    
    
    # show output can be commented or deleted to not see error message
    if ShowVideoWhileProcessing:
        cv2.imshow('frame', frm)  # final frame
    # cv2.imshow('mask', fgmask)  # shape of the mouse as detected after processing

    if ShowSubstractedVideo:
        cv2.imshow("__", thresh)

    # Close session 
    # not working in notebook
    k = cv2.waitKey(5)  
    if k == 3:                # stop the output after 3 button presses
        break



decalage = 0
for i in range(0,length - delaiBetweenVideoExperiment): 
#for i in range(0,1000): #used for testing or small visionning
       
    #--------------------------------------------------
    # open & process frames & get animal position
    #--------------------------------------------------

    # get the current frame  
    ret, frm = cap.read()

    # resize the frame and the output 
    frm = cv2.resize(frm, (resolution), interpolation = cv2.INTER_AREA) 
    imgTrack = np.zeros_like(frm)

    # convert to grayscale and add gaussian blur
    frameGray = cv2.cvtColor(frm, cv2.COLOR_BGR2GRAY)  #can be commented if acquisition done in greyscale (it's the case when doing it with python script)
    kernelSize = (25,25)
    frameBlur = cv2.GaussianBlur(frameGray, kernelSize, 0) ## blurring removes details/noise

    # apply the background substraction
    thresh = fgbg.apply(frameBlur,learningRate=0.0004) #learning rate == value between 0 and 1 that indicates how fast the background model is learnt. 0 means that the background model is not updated at all, 1 means that the background model is completely reinitialized from the last frame.
    fgmask = cv2.resize(thresh, (resolution)) ## this is only used when display thresh

    # compute center of mass of what's left and get x, y of the centroid
    M = cv2.moments(thresh)
    if M['m00'] == 0:
        decalage = decalage +1
        continue
    # if i < index : 
    #     x = int(M['m10'] / M['m00'])
    #     y = int(M['m01'] / M['m00'])
    # else : 
        #dfi = i - index
    try :
        x = int(dfxyt.loc[dfxyt.index[i - decalage], 'xposition'])
        y = int(dfxyt.loc[dfxyt.index[i - decalage], 'yposition'])
    except : continue
    if _x == 0 and _y == 0:  #for the first frame now the previous frame take the current valu
        _x = x
        _y = y
        
    #Saves timestamp and position in arrays for later use / save to .csv
    t+=1/framerate
    time_save=np.append(time_save,t)
    x_pos=np.append(x_pos,x)
    y_pos=np.append(y_pos,y)

    

    # if dfxyt.loc[dfxyt.index[i], "xposition"] != x or dfxyt.loc[dfxyt.index[i], "yposition"] != y : 
    #     print("incoherent detection !")
    #     print( f" ligne {i}, expected xvalue : {dfxyt.loc[dfxyt.index[i], 'xposition']}, obtained value : {x} \n expected yvalue {dfxyt.loc[dfxyt.index[i], 'yposition']}, obtained : {y}")
    #     #break

    
    # plot a red dot at x, y 
    frm = cv2.circle(frm, (x,y), 7, (0,   0,   255), -1)
    thresh = cv2.circle(thresh, (x,y), 7, (255,   255,   255), -1)
    # plot the line between 2 successive frames and add it to frame
    imgTrack = cv2.addWeighted(np.zeros_like(imgTrack), 0.85, cv2.line(imgTrack, (x,y), (_x,_y), (255, 127, int(cap.get(cv2.CAP_PROP_POS_AVI_RATIO) * 255)), 2, cv2.LINE_AA), 0.98, 0.)
    frm = cv2.addWeighted(frm, 0.4, imgTrack, 1.0, 0.)


    ## number of the frame that is increasing on the upper right corner of the tracked video
    frame_number = cap.get(cv2.CAP_PROP_POS_FRAMES)
    if expert_mode : 
        cv2.putText(frm, str(frame_number), (400, 30), fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 0.4, color = (0,255,255))


    
    # Determine  in which quandrant/zone  is the mouse 
    # frame of size (resolution*resolution), so quadrants are of size (resolution/2  * resolution/2)
    # we determine the width of the trapezes with TRAPEZE_SIZE

    if x <= resolution[0]/2 and y <= resolution[0]/2: #patch NW
        currentPatch = patchNW, 'NW'
        N, E, S, W = trapezes_from_patch(NWpatch_coords, TRAPEZE_SIZE)
    elif x >= resolution[0]/2 and y<=resolution[0]/2: #patch NE
        currentPatch = patchNE, 'NE'
        N, E, S, W = trapezes_from_patch(NEpatch_coords, TRAPEZE_SIZE)
    elif x <=resolution[0]/2 and y>=resolution[0]/2: #patch SW
        currentPatch = patchSW, 'SW'
        N, E, S, W = trapezes_from_patch(SWpatch_coords, TRAPEZE_SIZE)
    elif x >=resolution[0]/2 and y>=resolution[0]/2: #patch SE
        currentPatch = patchSE, 'SE'
        N, E, S, W = trapezes_from_patch(SEpatch_coords, TRAPEZE_SIZE)
    else: print("where is the mouse?", x, y)

    # Plot the current patch (rembmer frm is actual frame)
    cv2.polylines(frm,[currentPatch[0]],True,(255,0,0), thickness=3)
    cv2.polylines(frm,[np.array(N, np.int32).reshape((-1,1,2))],True,(255,0,0), thickness=3)
    cv2.polylines(frm,[np.array(E, np.int32).reshape((-1,1,2))],True,(255,0,0), thickness=3)
    cv2.polylines(frm,[np.array(S, np.int32).reshape((-1,1,2))],True,(255,0,0), thickness=3)
    cv2.polylines(frm,[np.array(W, np.int32).reshape((-1,1,2))],True,(255,0,0), thickness=3)


    # In the current patch, determine if the mouse is in one of the trapezes with points_in_polygon()
    # If yes, change color/open valve/etc.
    if points_in_polygon(N, [[x, y]]): 
        currentTrapeze = 'N'
    if points_in_polygon(E, [[x, y]]): 
        currentTrapeze = 'E'
    if points_in_polygon(S, [[x, y]]): 
        currentTrapeze = 'S'
    if points_in_polygon(W, [[x, y]]): 
        currentTrapeze = 'W'
    if not points_in_polygon(N, [[x, y]]) and not points_in_polygon(E, [[x, y]]) and not points_in_polygon(S, [[x, y]]) and not points_in_polygon(W, [[x, y]]): 
        currentTrapeze = 'none'
        type_of_turn = "niao"

    #--------------------------------------------------
    # detect when zone changes. if change follows task rule, deliver reward
    #--------------------------------------------------
        
    # detect frame when animal changes trapeze and display it
    if str(_currentTrapeze) != str(currentTrapeze):
        change2 = datetime.datetime.now()
        changedTrapeze = True 
    else:
        changedTrapeze = False
    # if (datetime.datetime.now() - change2).total_seconds() < 1 and expert_mode: # this is just to display during one second when the animal changed trapeze

    cv2.putText(frm, str(df.loc[df.index[current_line_turn_info], "totalnberOfRewards"]) + "/" + str(indexDF), (50, 10), fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 0.4, color = (250,225,100))
    if expert_mode : 
        cv2.putText(frm, currentTrapeze + " " +str(df.loc[df.index[current_line_turn_info], "currentTrapeze"]), (50, 50), fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 0.4, color = (250,225,100)) # we always plot
    #the label of the currentTrapeze

    # Task rule
    # Deliver water when going clockwise or counterclockwise as defined in the first part fo the code
    # assign a number for each trapeze, if must go counterclockwise to get reward (3>2>1>0>3 it's -1/-1/-1/-1/-1)
    # return true if follows this sequence, else false (e.g arriving from other patch, going clockwise, skipping a trapeze)
    # 'none' trapeze (outside of the object) is set to 0.5 so it doesn't interfere with the comparisons

    if str(currentTrapeze) == 'none' or str(_currentTrapeze) == 'none':
        do_I_get_water = False
        conseq=np.append(conseq,0)
        
        
    elif str(currentPatch[1]) == good_object : #if mouse is in the good object
        if str(_currentTrapeze) == str(currentTrapeze) : #Mouse was in the same trapeze last frame
            do_I_get_water = False

          

        if changedTrapeze and str(_currentTrapeze) != 'none': # if changed trapeze

            
            if ShowVideoWhileProcessing or ShowSubstractedVideo:
                print('previous and current trapezes')
                print(_currentTrapeze,currentTrapeze)
                print('turn angle:')
                print(angle_between(cardinalvectors[currentTrapeze],cardinalvectors[_currentTrapeze]))
                print('is angle OK')
                print(angle_between(cardinalvectors[currentTrapeze],cardinalvectors[_currentTrapeze]) in direction)


            if angle_between(cardinalvectors[currentTrapeze],cardinalvectors[_currentTrapeze])not in direction:  # wrong turn
                do_I_get_water = False
                conseq=np.append(conseq,0)                
                btgo_c= np.append(btgo_c,btgo_c[-1]+1)
                btgo_t= np.append(btgo_t,t)
                type_of_turn = "gobt"
                
            if angle_between(cardinalvectors[currentTrapeze],cardinalvectors[_currentTrapeze]) in direction:  # ok turn
                count_tot+=1
                conseq=np.append(conseq,conseq[-1]+1)                
                gtgo_c= np.append(gtgo_c,gtgo_c[-1]+1)
                gtgo_t= np.append(gtgo_t,t)
             
                type_of_turn = "gogt"
                do_I_get_water = True
                #the animal did the correct turn, but depending on the version, it does not necessarly get a reward
                if reward_mode != "remaining" : 
                    type_of_turn = df.loc[df.index[current_line_turn_info + 1], "typeOfTurn"]
                    do_I_get_water = df.loc[df.index[current_line_turn_info + 1 ], "Rewarded"]
                
            if angle_between(cardinalvectors[currentTrapeze],cardinalvectors[_currentTrapeze])==180:  # opposite side DavQ: useless
                do_I_get_water = False
                conseq=np.append(conseq,0)
                
                

            #change to the next line of turns_info, as well as good_object and the direction
            if i >= index :
                
                #remaining_rewards = str(df.iat[current_line_turn_info, 12] - df.iat[current_line_turn_info, 11])
                current_line_turn_info += 1
                current_object = ongoingRewardedObject[current_line_turn_info][0] ; current_direction = ongoingRewardedDirection[current_line_turn_info][1:]
                do_I_get_water, remaining_rewards, type_of_turn = reward_trapeze_change(current_line_turn_info, df, do_I_get_water, reward_mode, remaining_rewards, type_of_turn)
                indexDF += 1 
                
                if current_line_turn_info > last_line : 
                    current_line_turn_info = last_line
                    direction = ongoingRewardedDirection[last_line][1:] ;  good_object = ongoingRewardedObject[last_line][0]
                else : 
                    direction = ongoingRewardedDirection[current_line_turn_info + 1][1:] ;  good_object = ongoingRewardedObject[current_line_turn_info + 1][0]
                

    ## the animal is not around the correct object but can do correct and incorrect turns
    elif str(currentPatch[1]) != good_object :
        conseq=np.append(conseq,0)
        if str(_currentTrapeze) == str(currentTrapeze) : 
            do_I_get_water = False
                       
        if changedTrapeze: # if changed trapeze
            if angle_between(cardinalvectors[currentTrapeze],cardinalvectors[_currentTrapeze])not in direction:  # wrong turn
                do_I_get_water = False                                          
                btbo_c= np.append(btbo_c,btbo_c[-1]+1)
                btbo_t= np.append(btbo_t,t)               
                #text = "wrong turn, bad object " + str(currentPatch[-1]) + " "
                type_of_turn = "bobt"
                
            if angle_between(cardinalvectors[currentTrapeze],cardinalvectors[_currentTrapeze]) in direction:  # ok turn DavQ we could change to say good turn bad of object
                do_I_get_water = False
                gtbo_c= np.append(gtbo_c,gtbo_c[-1]+1)
                gtbo_t= np.append(gtbo_t,t)           
                #text = "good turn, bad object " + str(currentPatch[-1]) +" "
                type_of_turn = "bogt"
                
            if angle_between(cardinalvectors[currentTrapeze],cardinalvectors[_currentTrapeze])==180:  # opposite side
                do_I_get_water = False
                #text = "skipped one"

            #change to the next line of turns_info, as well as good_object and the direction
            if i >= index :

                current_line_turn_info += 1
                current_object = ongoingRewardedObject[current_line_turn_info][0] ; current_direction = ongoingRewardedDirection[current_line_turn_info][1:]
                do_I_get_water, remaining_rewards, type_of_turn = reward_trapeze_change(current_line_turn_info, df, do_I_get_water, reward_mode, remaining_rewards, type_of_turn)
                indexDF += 1 
                
                if current_line_turn_info > last_line : 
                    current_line_turn_info = last_line
                    direction = ongoingRewardedDirection[last_line][1:] ;  good_object = ongoingRewardedObject[last_line][0]
                else : 
                    direction = ongoingRewardedDirection[current_line_turn_info + 1][1:] ;  good_object = ongoingRewardedObject[current_line_turn_info + 1][0]



    if currentPatch[1] != _currentPatch[1] and exploration_time > 0:
        if not exploring : 
            exploring = True
            exploration_timer = time_save[-1]
        if exploration_timer != 0 and (time_save[-1] - exploration_timer) > exploration_time : 
            duration_last_exploration = round(time_save[-1] - exploration_timer, 2)
            exploration_timer = 0
                
    if do_I_get_water and exploring and exploration_timer == 0: #end the exploration
        exploring = False
        



    #determine the color of the trapeze the mouse is in depending on the type of the last turn


    couleur = couleur_turn[type_of_turn]
    
    # In the current patch, determine if the mouse is in one of the trapezes with points_in_polygon()
    # If yes, change color/open valve/etc.
    if currentTrapeze == 'N': 
        cv2.polylines(frm,[np.array(N, np.int32).reshape((-1,1,2))],True, couleur, thickness=3)
        
    elif currentTrapeze == 'E': 
        cv2.polylines(frm,[np.array(E, np.int32).reshape((-1,1,2))],True, couleur, thickness=3)
        
    elif currentTrapeze == 'S': 
        cv2.polylines(frm,[np.array(S, np.int32).reshape((-1,1,2))],True, couleur, thickness=3)
        
    elif currentTrapeze == 'W': 
        cv2.polylines(frm,[np.array(W, np.int32).reshape((-1,1,2))],True, couleur, thickness=3)
          

    #plot the position of the waterports
    for w in waterports: cv2.circle(frm, waterports[w], 3, (0, 0, 255), thickness=-1, lineType=cv2.LINE_AA)


    if expert_mode : cv2.putText(frm, str(indexDF + 2) , (50, 30), fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 0.4, color = (250,225,100)) #write the text

    if exploring and exploration_time > 0 and exploration_timer != 0: 
        cv2.putText(frm, f"exploring since {round(time_save[-1] - exploration_timer, 2)}s", (150, 10), fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 0.4, color = (250,225,100))
    if duration_last_exploration != 0 and exploration_time > 0 : 
        cv2.putText(frm, f"last exploration {duration_last_exploration}s", (150, 50), fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 0.4, color = (250,225,100))


    
    #if the mouse enter the new rewarded object, show its informations instead of those of the last turn
    if currentPatch[1] != _currentPatch[1] and currentPatch[1] == good_object : 
        current_direction = direction
        current_object = good_object
        if current_line_turn_info < last_line : 
            if reward_mode == "remaining" : 
                if df.loc[df.index[current_line_turn_info + 1], "nberOfConsecRewards"] == 1 :#if this will be the first reward obtained, then the current number of remaining rewards is the maximum
                    remaining_rewards = str(df.loc[df.index[current_line_turn_info + 1], "maxNberOfConsecRewards"])
            elif reward_mode == "fixed" : 
                remaining_rewards = df_session.loc[df_session.index[0], "proba" + ongoingRewardedObject[current_line_turn_info + 1][0]]
            elif reward_mode == "linear_increase" : 
                remaining_rewards = 0
    

    if current_direction[0] == 90 : around = 1
    else : around = 0
    #draw an arrow indicating the direction of the last turn and print the number of reward remaining
    if type_of_turn != "expl"  and not (type_of_turn == "niao" and df.loc[df.index[current_line_turn_info + 1], "typeOfTurn"] == "expl"): 
        if reward_mode == "remaining" or current_object != "none" :
            if len(current_direction) == 1 : 
                frm = cv2.polylines(frm, [arrows_body[current_object][around]],False, color= (255, 255, 255), thickness= 3)# draw the body of the arrow
                frm = cv2.line(frm, list(arrows_body[current_object][around][0][0]), list(arrows_body[current_object][around][0][0] + (-5, 5)), color = (255, 255, 255), thickness= 3)#these two draw the head of the arrow
                frm = cv2.line(frm, list(arrows_body[current_object][around][0][0]), list(arrows_body[current_object][around][0][0] + (5, 5)), color = (255, 255, 255), thickness= 3)
            if  str(currentPatch[1]) == current_object :
                cv2.putText(frm, str(remaining_rewards), reward_position[current_object], fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 0.4, color = (0,255,255))
            if i == 770 :
                print(1)
        

    if do_I_get_water == True :  # save frame when reward was delivered
        rew = np.append(rew,1)
    else :
        rew = np.append(rew,0)
    
    # end of the operation on the current frame. We now update _x and _y such as for the next frame the current frame is the previous one 
    _x = x
    _y = y
    _currentPatch = currentPatch
    _currentTrapeze = currentTrapeze
    _rewarded = rewarded

    
    
    # print computer_mouse position when click on frame    
    # cv2.setMouseCallback("frame", mousePoints)

    out.write(frm) #saves frame to output video
    
    
    # show output can be commented or deleted to not see error message
    if ShowVideoWhileProcessing:
        cv2.imshow('frame', frm)  # final frame
    # cv2.imshow('mask', fgmask)  # shape of the mouse as detected after processing

    if ShowSubstractedVideo:
        cv2.imshow("__", thresh)


    #Save the zone and trapeze in which the mouse is at each frame, will be useful to define the transition paths
    trapeze=np.append(trapeze,currentTrapeze)
    zone=np.append(zone,currentPatch[1])
    # Close session 
    # not working in notebook
    k = cv2.waitKey(5)  
    if k == 3:                # stop the output after 3 button presses
        break
    
#Release the input and output video and close the display windows
cap.release() 
out.release()
cv2.destroyAllWindows()



stop = timeit.default_timer() 
print('Time: ', stop - start)

#copy orginal video file in the processed folder

os.rename(input_video_path,final_Video_path) 



1
MOU3975_20230318-1737
2
/localdata/alicedata/MOU3975/MOU3975_20230318-1737
processing folder already exist
3
/localdata/alicedata/MOU3975/MOU3975_20230318-1737/MOU3975_20230318-1737.avi
4
/localdata/alicedata/MOU3975/MOU3975_20230318-1737/MOU3975_20230318-1737.avi
Video found


/tmp/ipykernel_20185/429423313.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ongoingRewardedObject'][i] = str([df['currentPatch'][i]])
